# Cutting Stocks Problem

## Crucial Libraries

In [1]:
from itertools import combinations
import random
import numpy as np
import matplotlib.pyplot as plt

## Seed Creation

In [2]:
seed_value = 12243493
random.seed(seed_value)
np.random.seed(seed_value)

In [9]:
data = {
    'stock_length': 40000,
    'demand_lengths': [1200, 1500, 800, 1000, 750, 2000, 650, 900, 500, 1300],
    'demand_quantities': [20, 30, 15, 25, 10, 18, 22, 17, 35, 12],
}

# genetic algorithm parameters
population_size = 100
num_generations = 100
mutation_rate = 0.01

# Cellular Automata parameters
grid_size = (10, 10)
num_iterations = 100  # Number of iterations for evolution
mutation_probability = 0.03  # Probability of random mutation to maintain diversity
reset_probability = 0.05  # Probability to reset underperforming cells


## Genetic Algorithms

In [10]:
def initialize_population(pop_size, demand_lengths, stock_length):
    population = []
    for _ in range(pop_size):
        individual = []
        remaining_length = stock_length
        for length in demand_lengths:
            max_count = remaining_length // length
            count = random.randint(0, max_count)
            individual.append(count)
            remaining_length -= count * length
        population.append(individual)
    return population

def fitness(individual, demand_lengths, demand_quantities, stock_length):
    total_used = sum([ind * length for ind, length in zip(individual, demand_lengths)])
    waste = stock_length - total_used

    demand_fulfillment = sum(min(ind, demand) for ind, demand in zip(individual, demand_quantities))
    penalty = sum(max(ind - demand, 0) for ind, demand in zip(individual, demand_quantities))
    fitness_score = (demand_fulfillment - waste) - penalty
    if total_used > stock_length:
        return 0

    return max(fitness_score, 0)

def selection(population, fitness_scores):
    min_fitness = min(fitness_scores)
    if min_fitness < 0:
        fitness_scores = [score - min_fitness + 1 for score in fitness_scores]

    total_fitness = sum(fitness_scores)
    selection_probs = [score / total_fitness for score in fitness_scores]
    selected_index = np.random.choice(range(len(population)), p=selection_probs)
    return population[selected_index]

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual, demand_lengths, stock_length):
    if random.random() < mutation_rate:
        index = random.randint(0, len(individual) - 1)
        remaining_length = stock_length - sum([ind * length for ind, length in zip(individual, demand_lengths)])

        max_mutation = max(remaining_length // demand_lengths[index] if demand_lengths[index] != 0 else 0, 0)

        if max_mutation > 0:
            mutation_value = random.randint(-1, max_mutation)
            individual[index] = max(0, individual[index] + mutation_value)  # Ensure non-negative values

def genetic_algorithm(data, pop_size, generations):
    demand_lengths = data['demand_lengths']
    demand_quantities = data['demand_quantities']
    stock_length = data['stock_length']

    population = initialize_population(pop_size, demand_lengths, stock_length)

    for generation in range(generations):
        fitness_scores = [fitness(individual, demand_lengths, demand_quantities, stock_length) for individual in population]
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = selection(population, fitness_scores)
            parent2 = selection(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1, demand_lengths, stock_length)
            mutate(child2, demand_lengths, stock_length)
            new_population.extend([child1, child2])

        population = new_population

    best_individual = max(population, key=lambda ind: fitness(ind, demand_lengths, demand_quantities, stock_length))
    best_fitness = fitness(best_individual, demand_lengths, demand_quantities, stock_length)

    return best_individual, best_fitness

best_solution, best_score = genetic_algorithm(data, population_size, num_generations)

print("Best Solution:", best_solution)
print("Best Score (Demand Fulfillment - Waste):", best_score)

Best Solution: [15, 5, 0, 3, 10, 2, 0, 0, 0, 0]
Best Score (Demand Fulfillment - Waste): 35


## Cellular Automata

In [15]:
def initialize_grid(grid_size, demand_lengths, stock_length):
    grid = []
    for _ in range(grid_size[0]):
        row = []
        for _ in range(grid_size[1]):
            pattern = []
            remaining_length = stock_length
            for length in demand_lengths:
                max_count = remaining_length // length
                count = random.randint(1, max(1, max_count)) if max_count > 0 else 1
                pattern.append(count)
                remaining_length -= count * length
            row.append(pattern)
        grid.append(row)
    return grid
def fitness(pattern, demand_lengths, demand_quantities, stock_length):
    total_used = sum([p * l for p, l in zip(pattern, demand_lengths)])
    waste = max(stock_length - total_used, 0)

    demand_fulfillment = sum(min(p, q) for p, q in zip(pattern, demand_quantities))

    unmet_penalty = sum(1 for p, q in zip(pattern, demand_quantities) if p < q)

    if total_used > stock_length:
        return 0

    return demand_fulfillment - 0.5 * waste - unmet_penalty

def mutate(individual, demand_lengths, stock_length):
    if random.random() < mutation_rate:
        index = random.randint(0, len(individual) - 1)
        remaining_length = stock_length - sum([ind * length for ind, length in zip(individual, demand_lengths)])

        max_mutation = max(remaining_length // demand_lengths[index] if demand_lengths[index] != 0 else 0, 0)

        mutation_value = random.randint(-1, max_mutation) if max_mutation > 0 else -1
        individual[index] = max(0, individual[index] + mutation_value)  # Ensure non-negative values

def update_grid(grid, demand_lengths, demand_quantities, stock_length, mutation_probability, reset_probability, iteration, num_iterations):
    current_reset_probability = reset_probability * (1 - iteration / num_iterations)

    new_grid = []
    for i, row in enumerate(grid):
        new_row = []
        for j, cell in enumerate(row):
            if random.random() < current_reset_probability:
                new_pattern = initialize_grid((1, 1), demand_lengths, stock_length)[0][0]
            else:
                neighbors = []
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    ni, nj = i + dx, j + dy
                    if 0 <= ni < len(grid) and 0 <= nj < len(grid[0]):  # Ensure indices are within bounds
                        neighbors.append(grid[ni][nj])
                current_fitness = fitness(cell, demand_lengths, demand_quantities, stock_length)
                best_neighbor = max(neighbors, key=lambda n: fitness(n, demand_lengths, demand_quantities, stock_length))
                best_neighbor_fitness = fitness(best_neighbor, demand_lengths, demand_quantities, stock_length)

                if best_neighbor_fitness > current_fitness:
                    new_pattern = best_neighbor[:]
                else:
                    new_pattern = cell[:]

                if random.random() < mutation_probability:
                    index = random.randint(0, len(new_pattern) - 1)
                    remaining_length = stock_length - sum([p * l for p, l in zip(new_pattern, demand_lengths)])
                    max_count = remaining_length // demand_lengths[index] if demand_lengths[index] != 0 else 0
                    mutation_value = random.randint(-max_count, max_count) if max_count > 0 else -1
                    new_pattern[index] = max(1, new_pattern[index] + mutation_value)

            new_row.append(new_pattern)
        new_grid.append(new_row)
    return new_grid

def cellular_automata(data, grid_size, num_iterations, mutation_probability, reset_probability):
    demand_lengths = data['demand_lengths']
    demand_quantities = data['demand_quantities']
    stock_length = data['stock_length']

    grid = initialize_grid(grid_size, demand_lengths, stock_length)
    print("Initial Grid with Fitness Scores:")
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            print(f"Cell ({i},{j}) - Pattern: {cell}, Fitness: {fitness(cell, demand_lengths, demand_quantities, stock_length)}")

    for iteration in range(num_iterations):
        if iteration % 10 == 0:
            print(f"\nIteration {iteration + 1}")
            for i, row in enumerate(grid):
                for j, cell in enumerate(row):
                    print(f"Cell ({i},{j}) - Pattern: {cell}, Fitness: {fitness(cell, demand_lengths, demand_quantities, stock_length)}")

        grid = update_grid(grid, demand_lengths, demand_quantities, stock_length, mutation_probability, reset_probability, iteration, num_iterations)

    best_patterns = []
    for row in grid:
        for pattern in row:
            if fitness(pattern, demand_lengths, demand_quantities, stock_length) > 0:
                best_patterns.append(pattern)

    total_fulfillment = [0] * len(demand_lengths)
    total_waste = 0
    for pattern in best_patterns:
        total_used = sum([p * l for p, l in zip(pattern, demand_lengths)])
        waste = stock_length - total_used
        total_waste += max(0, waste)  # Only add positive waste

        for i in range(len(demand_lengths)):
            total_fulfillment[i] += min(pattern[i], demand_quantities[i])

    total_fulfillment = [min(f, q) for f, q in zip(total_fulfillment, demand_quantities)]

    return best_patterns, total_fulfillment, total_waste

best_patterns, total_fulfillment, total_waste = cellular_automata(data, grid_size, num_iterations, mutation_probability, reset_probability)

print("\nBest Patterns Found:")
for pattern in best_patterns:
    print(pattern)
print("Total Demand Fulfillment:", total_fulfillment)
print("Total Waste:", total_waste)


Initial Grid with Fitness Scores:
Cell (0,0) - Pattern: [16, 11, 3, 1, 1, 1, 1, 1, 1, 1], Fitness: 0
Cell (0,1) - Pattern: [14, 2, 25, 1, 1, 1, 1, 1, 1, 1], Fitness: 0
Cell (0,2) - Pattern: [15, 6, 9, 5, 1, 1, 1, 1, 1, 1], Fitness: 0
Cell (0,3) - Pattern: [16, 1, 10, 6, 5, 1, 1, 1, 1, 1], Fitness: 0
Cell (0,4) - Pattern: [1, 8, 5, 4, 17, 2, 3, 1, 1, 1], Fitness: 0
Cell (0,5) - Pattern: [6, 16, 2, 7, 1, 1, 1, 1, 1, 1], Fitness: 0
Cell (0,6) - Pattern: [24, 3, 7, 1, 1, 1, 1, 1, 1, 1], Fitness: 0
Cell (0,7) - Pattern: [13, 5, 15, 3, 1, 1, 1, 1, 1, 1], Fitness: 0
Cell (0,8) - Pattern: [15, 7, 8, 2, 2, 1, 1, 1, 1, 1], Fitness: 0
Cell (0,9) - Pattern: [1, 2, 2, 33, 1, 1, 1, 1, 1, 1], Fitness: 0
Cell (1,0) - Pattern: [3, 2, 10, 14, 14, 1, 1, 1, 1, 1], Fitness: 0
Cell (1,1) - Pattern: [13, 11, 1, 2, 1, 2, 1, 1, 1, 1], Fitness: 0
Cell (1,2) - Pattern: [30, 1, 1, 1, 1, 1, 1, 1, 1, 1], Fitness: 0
Cell (1,3) - Pattern: [14, 2, 10, 4, 2, 1, 3, 2, 1, 1], Fitness: 0
Cell (1,4) - Pattern: [12, 1, 21, 

## Ant Colony Algorithm